In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/final_pesu_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,0.0,0.7096,Positive,5
1,1,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,0.0,0.2960,Positive,4
2,2,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,0.0,0.5994,Positive,5
3,3,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,0.0,0.7096,Positive,5
4,4,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,0.0,0.2960,Positive,4
5,5,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,0.0,0.5994,Positive,5
6,6,Just posted a photo @ PES University https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0,0,0.0,0.0000,Neutral,3
7,7,I’m am genuinely thinking to boycott EA now. T...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0,1,0.0,0.1525,Positive,3
8,8,Be sure to come to PES University on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0,2,0.0,0.3802,Positive,4
9,9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"['dscpesu', 'google', 'googlecloudpic']",dscpesu,0,3,1.0,-0.2732,Negative,2


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000
1,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000
2,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996
3,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000
4,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000
5,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996
6,Just posted a photo @ PES University https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000
7,I’m am genuinely thinking to boycott EA now. T...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0.0,1.0,0.0,0.1525,Positive,3,0.000000,0.001996
8,Be sure to come to PES University on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0.0,2.0,0.0,0.3802,Positive,4,0.000000,0.003992
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"['dscpesu', 'google', 'googlecloudpic']",dscpesu,0.0,3.0,1.0,-0.2732,Negative,2,0.000000,0.005988


In [6]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [7]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [8]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [9]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [10]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea..."
1,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w..."
2,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,..."
3,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea..."
4,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w..."
5,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,..."
6,just posted a photo @ pes university https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Just posted a photo PES University …,"[posted, photo, pes, university]"
7,i’m am genuinely thinking to boycott ea now. t...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0.0,1.0,0.0,0.1525,Positive,3,0.000000,0.001996,0.079840,I’m am genuinely thinking to boycott EA now Th...,"[genuinely, thinking, boycott, ea, mean, going..."
8,be sure to come to pes university on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0.0,2.0,0.0,0.3802,Positive,4,0.000000,0.003992,0.159681,Be sure to come to PES University on the 20th …,"[sure, come, pes, university, 20th]"
9,drop in to pes university on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"['dscpesu', 'google', 'googlecloudpic']",dscpesu,0.0,3.0,1.0,-0.2732,Negative,2,0.000000,0.005988,0.239521,Drop in to PES University on the 20th to meet ...,"[drop, pes, university, 20th, meet, google, em..."


In [11]:
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea...","[(bangalore, RB), (youth, JJ), (bhaarat, NN), ..."
1,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w...","[(join, NN), (edtech, JJ), (conference, NN), (..."
2,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,...","[(looks, NNS), (interesting, VBG), (join, NN),..."
3,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea...","[(bangalore, RB), (youth, JJ), (bhaarat, NN), ..."
4,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w...","[(join, NN), (edtech, JJ), (conference, NN), (..."
5,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,...","[(looks, NNS), (interesting, VBG), (join, NN),..."
6,just posted a photo @ pes university https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Just posted a photo PES University …,"[posted, photo, pes, university]","[(posted, VBN), (photo, NN), (pes, NNS), (univ..."
7,i’m am genuinely thinking to boycott ea now. t...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0.0,1.0,0.0,0.1525,Positive,3,0.000000,0.001996,0.079840,I’m am genuinely thinking to boycott EA now Th...,"[genuinely, thinking, boycott, ea, mean, going...","[(genuinely, RB), (thinking, VBG), (boycott, N..."
8,be sure to come to pes university on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0.0,2.0,0.0,0.3802,Positive,4,0.000000,0.003992,0.159681,Be sure to come to PES University on the 20th …,"[sure, come, pes, university, 20th]","[(sure, JJ), (come, VBP), (pes, NNS), (univers..."
9,drop in to pes university on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"['dscpesu', 'google', 'googlecloudpic']",dscpesu,0.0,3.0,1.0,-0.2732,Negative,2,0.000000,0.005988,0.239521,Drop in to PES University on the 20th to meet ...,"[drop, pes, university, 20th, meet, google, em...","[(drop, NN), (pes, NNS), (university, NN), (20..."


In [12]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea...","[(bangalore, RB), (youth, JJ), (bhaarat, NN), ...","[bhaarat, chetty, year, computer, science, stu..."
1,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w...","[(join, NN), (edtech, JJ), (conference, NN), (...","[join, conference, july, university, day, rate..."
2,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,...","[(looks, NNS), (interesting, VBG), (join, NN),...","[looks, join, conference, july, university, da..."
3,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,Positive,5,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea...","[(bangalore, RB), (youth, JJ), (bhaarat, NN), ...","[bhaarat, chetty, year, computer, science, stu..."
4,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,Positive,4,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w...","[(join, NN), (edtech, JJ), (conference, NN), (...","[join, conference, july, university, day, rate..."
5,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,Positive,5,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,...","[(looks, NNS), (interesting, VBG), (join, NN),...","[looks, join, conference, july, university, da..."
6,just posted a photo @ pes university https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Just posted a photo PES University …,"[posted, photo, pes, university]","[(posted, VBN), (photo, NN), (pes, NNS), (univ...","[photo, pes, university]"
7,i’m am genuinely thinking to boycott ea now. t...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0.0,1.0,0.0,0.1525,Positive,3,0.000000,0.001996,0.079840,I’m am genuinely thinking to boycott EA now Th...,"[genuinely, thinking, boycott, ea, mean, going...","[(genuinely, RB), (thinking, VBG), (boycott, N...","[boycott, ea, mean, pes, money, things, univer..."
8,be sure to come to pes university on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0.0,2.0,0.0,0.3802,Positive,4,0.000000,0.003992,0.159681,Be sure to come to PES University on the 20th …,"[sure, come, pes, university, 20th]","[(sure, JJ), (come, VBP), (pes, NNS), (univers...","[pes, university]"
9,drop in to pes university on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"['dscpesu', 'google', 'googlecloudpic']",dscpesu,0.0,3.0,1.0,-0.2732,Negative,2,0.000000,0.005988,0.239521,Drop in to PES University on the 20th to meet ...,"[drop, pes, university, 20th, meet, google, em...","[(drop, NN), (pes, NNS), (university, NN), (20...","[drop, pes, university, meet, google, employee..."


In [13]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,bhaarat
1,chetty
2,year
3,computer
4,science
5,student
6,university
7,city
8,proud
9,developer


In [14]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,bhaarat,bhaarat,3
1,chetty,chetty,2
2,year,year,38
3,computer,computer,8
4,science,science,44
5,student,student,111
6,university,university,1073
7,city,city,33
8,proud,proud,3
9,developer,developer,7


In [15]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
6,university,university,1073
25,pes,pes,777
15,com,com,453
14,twitter,twitter,451
47,pic,pic,209
871,admission,admission,174
58,students,students,138
5,student,student,111
82,engineering,engineering,82
81,college,college,65


In [16]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [17]:
#list of irreleveant tweet topics
l=['bello','manchester','trump','fifa','minnesota','iowa','minneapolis']

In [18]:
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
108,pes conference 12-13 september manchester. key...,BPS_PES,1168965676196929536,2019-09-03 19:15:10,[],BPS_PES,2.0,3.0,0.0,0.0000,Neutral,3,0.022727,0.005988,1.603157,PES Conference 12-13 September Manchester Keyn...,"[pes, conference, 12, 13, september, mancheste...","[(pes, NNS), (conference, NN), (12, CD), (13, ...","[pes, conference, september, manchester, keyno...",1
113,the stage is set! \n\nstanbic ibtc; you grow f...,#BBNaija #BBNaija2020,1167619601091497984,2019-08-31 02:06:20,['StanbicIBTCpic'],Unibadan_efiwe,0.0,1.0,0.0,0.8356,Positive,5,0.000000,0.001996,0.079840,The Stage Is SET Stanbic IBTC You Grow Fair @S...,"[stage, set, stanbic, ibtc, grow, fair, stanbi...","[(stage, NN), (set, VBN), (stanbic, JJ), (ibtc...","[stage, ibtc, grow, stanbicibtc, venue, lectur...",1
119,pes conference 12-13 september manchester. key...,BPS_PES,1168965676196929536,2019-09-03 19:15:10,[],BPS_PES,2.0,3.0,0.0,0.0000,Neutral,3,0.022727,0.005988,1.603157,PES Conference 12-13 September Manchester Keyn...,"[pes, conference, 12, 13, september, mancheste...","[(pes, NNS), (conference, NN), (12, CD), (13, ...","[pes, conference, september, manchester, keyno...",1
124,the stage is set! \n\nstanbic ibtc; you grow f...,#BBNaija #BBNaija2020,1167619601091497984,2019-08-31 02:06:20,['StanbicIBTCpic'],Unibadan_efiwe,0.0,1.0,0.0,0.8356,Positive,5,0.000000,0.001996,0.079840,The Stage Is SET Stanbic IBTC You Grow Fair @S...,"[stage, set, stanbic, ibtc, grow, fair, stanbi...","[(stage, NN), (set, VBN), (stanbic, JJ), (ibtc...","[stage, ibtc, grow, stanbicibtc, venue, lectur...",1
243,congratulations to all the ieee r8 society cha...,Dr. Mona Ghassemian,1185834208813445120,2019-10-20 08:24:41,"['ieee', 'awardwinning', 'PES']",monagreenwich,3.0,5.0,0.0,0.9732,Positive,5,0.034091,0.009980,2.444656,Congratulations to all the IEEE R8 society cha...,"[congratulations, ieee, r8, society, chapter, ...","[(congratulations, NNS), (ieee, VBP), (r8, JJ)...","[congratulations, society, chapter, award, win...",1
246,photo at ieee section meeting dinner. from lef...,IEEE PES UKRI,1185205517662261248,2019-10-18 14:46:30,[],IEEE_PES_UKRI,1.0,4.0,0.0,0.0000,Neutral,3,0.011364,0.007984,1.001179,Photo at IEEE Section Meeting dinner From left...,"[photo, ieee, section, meeting, dinner, left, ...","[(photo, NN), (ieee, JJ), (section, NN), (meet...","[photo, section, meeting, dinner, e, audiche, ...",1
247,ieee uk & ireland section meeting at universit...,IEEE PES UKRI,1185203272514265088,2019-10-18 14:37:34,[],IEEE_PES_UKRI,1.0,4.0,0.0,0.9552,Positive,5,0.011364,0.007984,1.001179,IEEE UK Ireland Section Meeting at University ...,"[ieee, uk, ireland, section, meeting, universi...","[(ieee, NN), (uk, JJ), (ireland, NN), (section...","[ieee, ireland, section, meeting, university, ...",1
259,congratulations to all the ieee r8 society cha...,Dr. Mona Ghassemian,1185834208813445120,2019-10-20 08:24:41,"['ieee', 'awardwinning', 'PES']",monagreenwich,3.0,5.0,0.0,0.9732,Positive,5,0.034091,0.009980,2.444656,Congratulations to all the IEEE R8 society cha...,"[congratulations, ieee, r8, society, chapter, ...","[(congratulations, NNS), (ieee, VBP), (r8, JJ)...","[congratulations, society, chapter, award, win...",1
262,photo at ieee section meeting dinner. from lef...,IEEE PES UKRI,1185205517662261248,2019-10-18 14:46:30,[],IEEE_PES_UKRI,1.0,4.0,0.0,0.0000,Neutral,3,0.011364,0.007984,1.001179,Photo at IEEE Section Meeting dinner From left...,"[photo, ieee, section, meeting, dinner, left, ...","[(photo, NN), (ieee, JJ), (section, NN), (meet...","[photo, section, meeting, dinner, e, audiche, ...",1
263,ieee uk & ireland section meeting at universit...,IEEE PES UKRI,1185203272514265088,2019-10-18 14:37:34,[],IEEE_PES_UKRI,1.0,4.0,0.0,0.9552,Positive,5,0.011364,0.007984,1.001179,IEEE UK Ireland Sectio

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']
df_eng1=df[df.engagement==1]
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
df[df.engagement==1]=df_eng1

In [23]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [24]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [25]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,...,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea...","[(bangalore, RB), (youth, JJ), (bhaarat, NN), ...","[bhaarat, chetty, year, computer, science, stu...",0,0,5
1,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,...,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w...","[(join, NN), (edtech, JJ), (conference, NN), (...","[join, conference, july, university, day, rate...",0,0,4
2,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,...,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,...","[(looks, NNS), (interesting, VBG), (join, NN),...","[looks, join, conference, july, university, da...",0,1,5
3,"bangalore youth, bhaarat chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0.0,0.0,0.0,0.7096,...,0.000000,0.000000,0.000000,Bangalore youth Bhaarat Chetty a final year Co...,"[bangalore, youth, bhaarat, chetty, final, yea...","[(bangalore, RB), (youth, JJ), (bhaarat, NN), ...","[bhaarat, chetty, year, computer, science, stu...",0,0,5
4,join our edtech conference 11th/12th july at w...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0.0,0.0,0.0,0.2960,...,0.000000,0.000000,0.000000,Join our EdTech conference 11th/12th July at W...,"[join, edtech, conference, 11th, 12th, july, w...","[(join, NN), (edtech, JJ), (conference, NN), (...","[join, conference, july, university, day, rate...",0,0,4
5,this looks interesting: join our edtech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1.0,1.0,0.0,0.5994,...,0.011364,0.001996,0.761659,This looks interesting Join our EdTech confere...,"[looks, interesting, join, edtech, conference,...","[(looks, NNS), (interesting, VBG), (join, NN),...","[looks, join, conference, july, university, da...",0,1,5
6,just posted a photo @ pes university https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0.0,0.0,0.0,0.0000,...,0.000000,0.000000,0.000000,Just posted a photo PES University …,"[posted, photo, pes, university]","[(posted, VBN), (photo, NN), (pes, NNS), (univ...","[photo, pes, university]",0,0,3
7,i’m am genuinely thinking to boycott ea now. t...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0.0,1.0,0.0,0.1525,...,0.000000,0.001996,0.079840,I’m am genuinely thinking to boycott EA now Th...,"[genuinely, thinking, boycott, ea, mean, going...","[(genuinely, RB), (thinking, VBG), (boycott, N...","[boycott, ea, mean, pes, money, things, univer...",0,0,3
8,be sure to come to pes university on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0.0,2.0,0.0,0.3802,...,0.000000,0.003992,0.159681,Be sure to come to PES University on the 20th …,"[sure, come, pes, university, 20th]","[(sure, JJ), (come, VBP), (pes, NNS), (univers...","[pes, university]",0,1,5
9,drop in to pes university on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"['dscpesu', 'google', 'googlecloudpic']",dscpesu,0.0,3.0,1.0,-0.2732,...,0.000000,0.005988,0.239521,Drop in to PES University on the 20th to meet ...,"[drop, pes, university, 20th, meet, google, em...","[(drop, NN), (pes, NNS), (university, NN), (20...","[drop, pes, university, meet, google, employee...",0,1,3


In [26]:
df.to_csv('../../Datasets/Twitter/PESUWeightedSentiment.csv')